In [0]:
%sql
-- 1. Bảng RAW: orders_raw (streaming từ parquet)

CREATE OR REFRESH STREAMING LIVE TABLE orders_raw
COMMENT "The raw orders data, ingested from parquet files."
AS SELECT * 
FROM cloud_files("${datasets.path}/orders", "parquet", map("cloudFiles.inferColumnTypes", "true"));

-- 2. Bảng RAW: customers (streaming từ JSON)

CREATE OR REFRESH STREAMING LIVE TABLE customers_raw
COMMENT "The raw customers data, ingested from JSON files."
AS SELECT * 
FROM cloud_files("${datasets.path}/customers", "json", map("cloudFiles.inferColumnTypes", "true"));

-- 3. Bảng cleaned: orders_cleaned

CREATE OR REFRESH LIVE TABLE orders_cleaned
COMMENT "Cleaned orders data with non-null order_id and positive quantity"
AS SELECT * 
FROM LIVE.orders_raw
WHERE order_id IS NOT NULL AND quantity > 0;

-- 4. Bảng final: enriched_orders

CREATE OR REFRESH LIVE TABLE enriched_orders
COMMENT "Join orders with customer data"
AS SELECT 
  o.order_id,
  o.customer_id,
  c.name AS customer_name,
  o.quantity,
  o.price,
  o.order_timestamp
FROM LIVE.orders_cleaned o
LEFT JOIN LIVE.customers_raw c
ON o.customer_id = c.customer_id;
